In [32]:
import pandas as pd
from flags import FLAGS, unparsed
import numpy as np
from db.conn_db import db,cursor,engine,truncate_table,data_from_mysql,dev_id_train
import operator  
from functools import reduce

In [26]:
deviceid_train=dev_id_train()

In [15]:
deviceid_packages= pd.read_csv(FLAGS.file_path +'deviceid_packages.csv',)

In [24]:
def app_list(text):
    app_list=text.split('|')
#        print (app_list)
    return app_list
deviceid_packages['app_list']=deviceid_packages['add_id_list'].apply(lambda line:app_list(line)).tolist()

In [27]:
deviceid_train=pd.merge(deviceid_train,deviceid_packages,on=['device_id'],how='left') 

In [28]:
deviceid_train.shape

(50000, 6)

In [29]:
deviceid_train.info()
deviceid_packages=deviceid_train.fillna(-1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 6 columns):
age            50000 non-null int64
device_id      50000 non-null object
n_class        50000 non-null int64
sex            50000 non-null int64
add_id_list    50000 non-null object
app_list       50000 non-null object
dtypes: int64(3), object(3)
memory usage: 2.7+ MB


In [30]:
app_1=deviceid_train.ix[deviceid_train['sex'].values==1,'app_list'].tolist()
app_2=deviceid_train.ix[deviceid_train['sex'].values==2,'app_list'].tolist()

/home/zhijiehuang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [39]:
def difference_list(app_list):
    import copy
    retlist=[]
    for i in range(0,len(app_list)):
        
        alist=app_list[i]
        tlist=copy.deepcopy(app_list)
        tlist.remove(alist)
        blist=reduce(operator.add, tlist)
        clist=list(set(alist).difference(set(blist)))
        retlist.append(clist)
    return retlist

In [42]:
def app_dict_get(app_list,app_dict):
    for x in app_list:
        if x in app_dict:
            return app_dict.get(x,0)
    

In [47]:
deviceid_train=deviceid_train.fillna(-1)
apps_list=[]
all_app=reduce(operator.add, deviceid_packages.app_list.tolist())
train_app=reduce(operator.add, deviceid_train.app_list.tolist())
no_train_app=list(set(all_app).difference(set(train_app)))
for i in range(0,23):
    type_list1=deviceid_train.ix[deviceid_train['n_class'].values==i,'app_list'].tolist()
    
    if len(type_list1)<1:
        apps_list.append([])
    else:
        type_list1=reduce(operator.add, type_list1)
        apps_list.append(type_list1)

diff_list=difference_list(apps_list)
app_dict={}
for i,x in enumerate(diff_list):
    for li in x:
        app_dict[li]=i




/home/zhijiehuang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [48]:
for i in range(0,23):
    clist=list(set(apps_list[i]).difference(set(diff_list[i]+no_train_app)))
    def c(a,b):
        ert=(a in b)
        return ert
    
    for x in clist:
        a=list(map(c,x,deviceid_train['app_list']))
        filte1=np.logical_and(deviceid_train.n_class==i,a)
        print(filte1)
        filte2=np.logical_and(a,True)
        app_dict[x]=deviceid_train.ix[filte1,'app_list'].shape[0]/deviceid_train.ix[filte2,'app_list'].shape[0]
for x in no_train_app:
    typeno_dict[x]=0
deviceid_packages['app1_w']=deviceid_packages['app_list'].apply(lambda x:app_dict_get(x,app_dict))

In [50]:
deviceid_train=deviceid_train.fillna(-1)
apps_list=[]
all_app=reduce(operator.add, deviceid_packages.app_list.tolist())
train_app=reduce(operator.add, deviceid_train.app_list.tolist())
no_train_app=list(set(all_app).difference(set(train_app)))
for i in [1,2]:
    type_list1=deviceid_train.ix[deviceid_train['sex'].values==i,'app_list'].tolist()
    
    if len(type_list1)<1:
        apps_list.append([])
    else:
        type_list1=reduce(operator.add, type_list1)
        apps_list.append(type_list1)

diff_list=difference_list(apps_list)
app_dict={}
for i,x in enumerate(diff_list):
    for li in x:
        app_dict[li]=i
deviceid_packages['app2_w']=deviceid_packages['app_list'].apply(lambda x:app_dict_get(x,app_dict))

/home/zhijiehuang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [36]:
deviceid_brand['type_no'].value_counts()

721     7281
2067    4914
776     4612
295     3952
1074    3625
996     2846
1177    1657
2028    1539
602     1344
696     1332
293     1077
1216    1006
26       894
286      827
24       746
265      729
1622     712
1220     672
1617     627
610      573
764      562
1292     537
1476     533
1472     457
30       452
131      431
1534     404
688      402
577      398
20       389
        ... 
170        1
2025       1
1043       1
1833       1
980        1
1896       1
1960       1
2024       1
724        1
105        1
233        1
361        1
404        1
340        1
489        1
553        1
681        1
84         1
2069       1
873        1
1129       1
1811       1
1321       1
1513       1
1577       1
1427       1
1363       1
1769       1
1171       1
0          1
Name: type_no, Length: 2079, dtype: int64

In [49]:
deviceid_packages

,age,device_id,n_class,sex,add_id_list,app_list,app1_w
0,10,00009270c4ec26e1d76f5d86847009c9,10,1,1896072db9ce6406febfc17f681c2086|90cb852cf345e...,"[1896072db9ce6406febfc17f681c2086, 90cb852cf34...",NaN
1,10,00026d79a6f0955fc860947724e24765,10,1,c33b35d6254ad9c0c238233eb97a6c60,[c33b35d6254ad9c0c238233eb97a6c60],NaN
2,5,0002e3afb8146bc08e40575e45f0eca6,5,1,1896072db9ce6406febfc17f681c2086|07e967d75aab2...,"[1896072db9ce6406febfc17f681c2086, 07e967d75aa...",NaN
3,7,0007f90fc1f2da95e0b8c307e1598d21,18,2,97d0422a3317b1929926dc90cda4fc53|1896072db9ce6...,"[97d0422a3317b1929926dc90cda4fc53, 1896072db9c...",18.0
4,10,000b216af4bf2cb7d8fe3cd66ddd03a8,10,1,1896072db9ce6406febfc17f681c2086|ae248d5abede6...,"[1896072db9ce6406febfc17f681c2086, ae248d5abed...",10.0
5,5,000ef93f440c2dec1da90f25227cef1c,16,2,37de3dfb9d3bc0a2264adf8802fef726|d977dd0e10adc...,"[37de3dfb9d3bc0a2264adf8802fef726, d977dd0e10a...",NaN
6,5,000efa9b64b19f672bf0963b76f3add9,5,1,1896072db9ce6406febfc17f681c2086|4b58ecb20fe0d...,"[1896072db9ce6406febfc17f681c2086, 4b58ecb20fe...",NaN
7,4,000f30a1b1adf884c3f5cf8cb57b6f5c,4,1,1896072db9ce6406febfc17f681c2086|91c5474aad059...,"[1896072db9ce6406febfc17f681c2086, 91c5474aad0...",NaN
8,2,000f3724e549428a979f20a9926be846,2,1,1896072db9ce6406febfc17f681c2086|86f9f299cdbc8...,"[1896072db9ce6406febfc17f681c2086, 86f9f299cdb...",NaN
9,8,00105d9dc897b8365cf8c2a806c9ab33,8,1,1896072db9ce6406febfc17f681c2086|4b58ecb20fe0d...,"[1896072db9ce6406febfc17f681c2086, 4b58ecb20fe...",NaN
